## Генераторы и итераторы.

In [25]:
import numpy as np


class RleSequence:
    def __init__(self, input_sequence):
        self.elements, self.amounts = self.encode_rle(input_sequence)
    
    def encode_rle(self, x):
        if len(x) > 0:
            idx_with_differences = np.append(np.where(np.diff(x) != 0)[0],
                                             np.argwhere(x == x[-1])[-1])
            return (x[idx_with_differences], np.diff(np.insert(idx_with_differences, 0, -1)))
        else:
            return None
    
    def decode_rle(self):
        decoded_sequence = np.zeros(np.sum(self.amounts), dtype=self.elements.dtype)
        indx = 0
        for i, element in enumerate(self.elements):
            for tmp in range(self.amounts[i]):
                decoded_sequence[indx] = element
                indx += 1
        return decoded_sequence
    
    def __iter__(self):
        self.curr_idx = 0
        self.curr_amount = 0
        return self
    
    def __next__(self):
        if self.curr_idx < len(self.elements):
            self.curr_amount += 1
            curr_elem = self.elements[self.curr_idx]
            if self.curr_amount >= self.amounts[self.curr_idx]:
                self.curr_amount = 0
                self.curr_idx += 1
            return curr_elem
        else:
            raise(StopIteration)
        
                
    def __getitem__(self, indx):
        if isinstance(indx, slice):
            start = indx.start
            stop = indx.stop
            step = indx.step
            if start is None:
                start = 0
            if stop is None:
                stop = np.sum(self.amounts)
            if step is None:
                step = 1
            return self.decode_rle()[start:stop:step]
        else:
            return self.decode_rle()[indx]
    
    def __contains__(self, target_elem):
        return target_elem in self.elements

In [26]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [27]:
%%memit
rle_seq = RleSequence(np.array([1, 1, 2, 1, 2, 2, 3, 3]))
for i in range(8):
    print(rle_seq[i])

1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
1
1
2
1
2
2
3
3
peak memory: 195.26 MiB, increment: 0.00 MiB


In [29]:
%%memit
np.random.seed(32)
tmp_array = np.random.randint(0, 3, 10 ** 6)

rle_seq = RleSequence(tmp_array)

sum_elements = 0
tmp = rle_seq[1:905005:2]
print(np.sum(tmp))

452902
peak memory: 213.56 MiB, increment: 18.29 MiB


## Task 2

In [62]:
from collections.abc import Iterable


def linearize(iterable_object):
    for elem in iterable_object:
        if isinstance(elem, Iterable):
            if isinstance(elem, str) & (len(elem) == 1):
                yield elem
            else:
                for elem2 in linearize(elem):
                    yield elem2
            continue       
        yield elem

In [63]:
my_list = []
for elem in linearize([4, 'mmp', [8, [15, 1], [[6]], [2, [3]]], range(4, 2, -1)]):
    my_list.append(elem)
my_list

[4, 'm', 'm', 'p', 8, 15, 1, 6, 2, 3, 4, 3]

## Task 3

In [176]:
import numpy as np


class BatchGenerator:
    def __init__(self, list_of_sequences, batch_size, shuffle=False):
        array_of_sequences = np.array(list_of_sequences)
        if shuffle:
            np.random.shuffle(array_of_sequences.T)

        self.array_of_sequences = array_of_sequences
        self.bs = batch_size
    
    def __next__(self):
        new_batch = np.hsplit(self.array_of_sequences)
        

In [76]:
import numpy as np


def BatchGenerator(list_of_sequences, batch_size, shuffle=False):
    array_of_sequences = np.array(list_of_sequences, dtype='O')
    if shuffle:
        np.random.shuffle(array_of_sequences.T)
    split_size = array_of_sequences.shape[1] // batch_size \
                                + (array_of_sequences.shape[1] % batch_size)
    for i in range(0, array_of_sequences.shape[1], batch_size):
        yield array_of_sequences[:, i:(i + batch_size)].tolist()

In [77]:
np.random.seed(42)
bg = BatchGenerator(list_of_sequences=[['84', 'u', 'w', '2', '5', '69', 'n', '27', 'k', 'k', '58', '96', 'j', '74'], ['1', '59', '24', 'z', '15', '70', '35', '79', '88', '99', 'w', '97', '52', '93']], batch_size=12, shuffle=False)
for e in bg:
	print(e)

[['84', 'u', 'w', '2', '5', '69', 'n', '27', 'k', 'k', '58', '96'], ['1', '59', '24', 'z', '15', '70', '35', '79', '88', '99', 'w', '97']]
[['j', '74'], ['52', '93']]


In [74]:
np.random.seed(42)
bg = BatchGenerator(list_of_sequences=[['12', '81', '48', '32', '75', '30'], ['64', '88', '89', '79', '68', 't'], ['11', '32', '74', '44', '58', '95'], ['79', '42', '88', 'v', '13', 'g'], ['40', 'x', '56', '67', '34', '1']], batch_size=5, shuffle=False)
for e in bg:
	print(e)

0
[['12', '81', '48', '32', '75'], ['64', '88', '89', '79', '68'], ['11', '32', '74', '44', '58'], ['79', '42', '88', 'v', '13'], ['40', 'x', '56', '67', '34']]
5
[['30'], ['t'], ['95'], ['g'], ['1']]


## Task 4

In [ ]:
class WordContextGenerator:
    def __init__(self, words, window_size):
        self.words = words
        self.window_size = window_size
    
    def __iter__(self):
        return self.words
    
    def __next__(self):
        new_pair = 

In [94]:
def WordContextGenerator(words, window_size):
    for i, word in enumerate(words):
        for left_word in words[max(i - window_size, 0):i]:
            yield word, left_word
        for right_word in words[i+1: i + window_size + 1]:
            yield word, right_word

In [95]:
s = ['мама', 'очень', 'хорошо', 'мыла', 'красивую', 'раму']

In [96]:
for elem in WordContextGenerator(s, window_size=2):
    print(elem)

('мама', 'очень')
('мама', 'хорошо')
('очень', 'мама')
('очень', 'хорошо')
('очень', 'мыла')
('хорошо', 'мама')
('хорошо', 'очень')
('хорошо', 'мыла')
('хорошо', 'красивую')
('мыла', 'очень')
('мыла', 'хорошо')
('мыла', 'красивую')
('мыла', 'раму')
('красивую', 'хорошо')
('красивую', 'мыла')
('красивую', 'раму')
('раму', 'мыла')
('раму', 'красивую')


# DECORATORS

## Task 5

In [99]:
import functools


def check_arguments(*args_for_decorator):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapper(*args):
            if len(args) < len(args_for_decorator):
                raise(TypeError)
            for i in range(len(args_for_decorator)):
                if not isinstance(args[i], args_for_decorator[i]):
                    raise(TypeError)    
            func(*args)
        return wrapper
    return actual_decorator

In [100]:
@check_arguments(int, int, int, str)
def test(k, l, m, w):
    return 0

In [101]:
test(1, 2, 3, '4')

In [103]:
# this is for similar behaviour in python 2 and python 3
from __future__ import print_function


@check_arguments(int)
def some_name(a):
    "Some doc."
    print(a)

print(some_name.__name__)
print(some_name.__doc__)

# don't forget to use functools.wraps

some_name
Some doc.


1 2 3
TypeError


## Task 6

In [52]:
import functools


def substitutive(func):
    
    @functools.wraps(func)
    def wrapper(*args):
        tmp_args = []
        args_for_save = []
        tmp_wrapper = None
        arguments_amount_real = len(args) + len(wrapper.__args_list__)
        if arguments_amount_real > func.__code__.co_argcount:
            raise TypeError
        try:
            for arg in wrapper.__args_list__:
                tmp_args.append(arg)
            for arg in args:
                tmp_args.append(arg)
            return func(*tmp_args)
        except:
            wrapper.__args_list__ = tmp_args[:]
            return wrapper
    wrapper.__args_list__ = []                
    return wrapper

In [53]:
@substitutive
def f(x, y, z):
    "Some doc"
    pass
try:
    f(1, 2)(1, 2)
except Exception as e:
    print(type(e).__name__)

TypeError


In [54]:
@substitutive
def f(x, y, z):
    "Some doc"
    print(x, y, z)

try:
    g = f(1, 2)
    g(3)
    g(4)
    g(5)
except Exception as e:
    print(type(e).__name__)

1 2 3
1 2 4
1 2 5


In [55]:
@substitutive
def f(a, b, c):
    print(a, b, c)

In [56]:
f(1, 2)(3)

1 2 3


In [57]:
f(1)(2)(3)

1 2 1


TypeError: 'NoneType' object is not callable

In [20]:
f(1, 2)

<function __main__.f(a, b, c)>

In [21]:
f(1, 2)(3)

<function __main__.f(a, b, c)>

In [58]:
@substitutive
def f(x, y, z):
    "Some doc"
    print(x, y, z)

try:
    f(1, 2, 3, 4)
except Exception as e:
    print(type(e).__name__)

TypeError


In [64]:
inspect

NameError: name 'inspect' is not defined